In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Access the API key
api_key = os.getenv("OPENROUTER_API_KEY")

# Access the base URL
base_url = os.getenv("OPENROUTER_BASE_URL")

# Show the API key
print(f"Your API key is: {api_key}")

In [ ]:
from llama_index.llms.openrouter import OpenRouter

# Initialize OpenRouter LLM with a custom base URL
llm = OpenRouter(
    api_key=api_key,
    model="google/gemini-2.0-flash-exp:free",
)

In [ ]:
from llama_index.core import SimpleDirectoryReader

# Load documents
documents = SimpleDirectoryReader("./data").load_data()

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# loads BAAI/bge-m3
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

In [ ]:
from llama_index.core import Settings

# Set the default LLM and embedding model
Settings.llm = llm
Settings.embed_model = embed_model

In [ ]:
import faiss

# dimensions for BAAI/bge-m3
d = 1024
faiss_index = faiss.IndexFlatL2(d)

In [ ]:
from llama_index.core import (
    SimpleDirectoryReader,
    load_index_from_storage,
    VectorStoreIndex,
    StorageContext,
)
from llama_index.vector_stores.faiss import FaissVectorStore
from IPython.display import Markdown, display

In [ ]:
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)

In [ ]:
# save index to disk
index.storage_context.persist()

In [ ]:
# load index from disk
vector_store = FaissVectorStore.from_persist_dir("./storage")
storage_context = StorageContext.from_defaults(
    vector_store=vector_store, persist_dir="./storage"
)
index = load_index_from_storage(storage_context=storage_context)

In [ ]:
# Use the query engine to get the response
query_engine = index.as_query_engine()
response = query_engine.query("Hello world")

print(str(response))